In [38]:
import pandas as pd
import numpy as np
import os
import dotenv
from weaviate.util import generate_uuid5


import re
from openai import AzureOpenAI



dotenv.load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')


# AZURE_OPEN_AI_ORGANIZATION = os.getenv('AZURE_OPEN_AI_ORGANIZATION')



In [9]:
import fsspec
from typing import Tuple, List
import pandas as pd

path = "/home/isma/repos/book/data-pipelines-with-airflow-2nd-ed/chapter13_genai/recipe_book/sample_recipes/2024-10-14"


def list_files_from_fs(path: str) -> List[str]:

    fs, base_path = fsspec.core.url_to_fs(path)

    return fs.ls(base_path)

files_to_process = list_files_from_fs(path)
files_to_process

['/home/isma/repos/book/data-pipelines-with-airflow-2nd-ed/chapter13_genai/recipe_book/sample_recipes/2024-10-14/empanadas-argentinas.txt',
 '/home/isma/repos/book/data-pipelines-with-airflow-2nd-ed/chapter13_genai/recipe_book/sample_recipes/2024-10-14/pavo-a-la-hawaiana.txt']

In [12]:
def get_minio_fs(path: str) -> Tuple[fsspec.spec.AbstractFileSystem, str]:   

    return fsspec.core.url_to_fs(path)

def append_content(files_to_process: List[str], path:str) -> pd.DataFrame:

    fs, base_path = get_minio_fs(f"{path}")
    content = []    

    for file in files_to_process:

        filename = file.split("/")[-1]

        with fs.open(f"{base_path}/{filename}", "rb") as file_:
            content.append({"filename": filename, "chunk": file_.read()})

    return pd.DataFrame(content)


In [35]:
def clean(df, content_col:str) -> pd.DataFrame:

    df = df.assign(
        **{content_col: lambda df: 
           df[content_col].astype(str)
           .str.replace("\n", " " )
            .str.strip()
        }
    )
   
    return df

In [41]:

df = (
    append_content(files_to_process, path)
    .pipe(clean, content_col="chunk")
    .assign( 
        document_sha = lambda df: [generate_uuid5(file) for file in df.filename],
        chunk_sha = lambda df: [generate_uuid5(file) for file in df.chunk],
    )
    .reset_index(drop=True)
    .loc[:, ["filename", "document_sha", "chunk_sha", "chunk"]]
)

df


,filename,document_sha,chunk_sha,chunk
0,empanadas-argentinas.txt,ec598303-9510-5d6e-9c5e-ccb43fcf8c4e,1e481049-3dcf-5027-87a2-420e8b8f4916,Empanadas argentinas Ingredientes Medidas 4 ...
1,pavo-a-la-hawaiana.txt,3303775d-98cc-5c62-adef-be3dce52c5a4,8624bb11-23e8-5c90-bf13-7aac35a210f1,Pavo a la hawaiana Ingredientes 1 taza de s...


In [11]:
client = AzureOpenAI(
  api_key = AZURE_OPENAI_API_KEY,  
  api_version = "2024-02-01",
  azure_endpoint = AZURE_OPENAI_ENDPOINT
)

In [12]:
def generate_embeddings(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding


In [13]:
data['vector'] = data["chunk"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002')) # model shou

In [23]:
type(df.chunk[0])

bytes